In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
save_path = "data/sleep_parameter_exploration"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [3]:
total_simulations = 125
sleep_time_threshold = 6.5
number_of_switches_threshold = 2.0

In [5]:
light_conditions = ["early", "late"]
zones_of_interest = [[21.0, 31.0], [23.0, 33.0]] # hours
for idx, light_condition in enumerate(light_conditions):
    # Calculate sleep time and number of switches
    zone_of_interest = zones_of_interest[idx]
    parameter_exploration = pd.DataFrame({})
    for idx in range(total_simulations):
        # Load simulation data
        data = np.load(f'data/sleep_parameter_exploration/{light_condition}_light_skeldon23_parameter_exploration_{idx}.npz', 
                    allow_pickle=True)
        simulation = data['result_data'].item()
        parameters = simulation["parameters"]
        model = simulation["simulation_model"]
        trajectory = simulation["trajectory"]
        time = trajectory.time
        dt = time[1] - time[0]
        time_of_interest_mask = (time >= zone_of_interest[0]) & (time <= zone_of_interest[1])
        time_of_interest = time[time_of_interest_mask]
        sleep = model.sleep_state
        sleep_of_interest = sleep[time_of_interest_mask]
        # number of switches
        number_of_switches = np.sum(np.abs(np.diff(sleep_of_interest)))
        # bedtime
        sleep_state_change = np.where(np.diff(sleep_of_interest) == 1)[0]
        bedtime = time_of_interest[sleep_state_change[0]]
        # wake up time
        sleep_state_change = np.where(np.diff(sleep_of_interest) == -1)[0]
        wake_up_time = time_of_interest[sleep_state_change[-1]]
        # sleep time
        sleep_time = wake_up_time - bedtime
        # final state of the simulation
        final_state = trajectory.states[-1, :]
        # DLMO
        dlmos = model.dlmos()
        result = pd.DataFrame({
            'mu': parameters['mu'],
            'Delta': parameters['Delta'],
            'chi': parameters['chi'],
            'sleep_time': sleep_time,
            'bedtime': bedtime,
            'dlmo': dlmos[1],
            'number_of_switches': number_of_switches,
            'x_f': final_state[0],
            'xc_f': final_state[1],
            'n_f': final_state[2],
            'H_f': final_state[3],
            'S_f': sleep[-1],
        }, index=[idx])
        parameter_exploration = pd.concat([parameter_exploration, result])


    desired_sleep_time = parameter_exploration['sleep_time'] >= sleep_time_threshold
    desired_number_of_switches = parameter_exploration['number_of_switches'] <= number_of_switches_threshold
    selected_parameters = parameter_exploration[desired_sleep_time & desired_number_of_switches]

    selected_parameters.index.name = 'simulation_id'
    selected_parameters.to_csv(f"{save_path}/{light_condition}_light_selected_sleep_parameters.csv")